In [ ]:
!pip install qdrant-client

In [ ]:
QDRANT_KEY = "i-XYrx9YHAWP8k2xRM_R9uwZlSIZWnfz5v4SIfREBqREYb7Hc83f8Q"
QDRANT_URI = "452250b2-c7ed-4c0e-bede-89cf5a18af83.us-east4-0.gcp.cloud.qdrant.io"
QDRANT_PORT = "6333"

In [ ]:
from qdrant_client import QdrantClient

try:
    # Kết nối tới Qdrant
    client = QdrantClient(
        url=f"https://{QDRANT_URI}",  # URI của Qdrant
        api_key=QDRANT_KEY           # API Key để xác thực
    )

    # In thông báo thành công
    print("Kết nối tới Qdrant thành công!")
    print(client)

except Exception as e:
    # In thông báo lỗi nếu kết nối thất bại
    print("Kết nối tới Qdrant thất bại!")
    print(f"Lỗi: {e}")

In [ ]:
# Lấy danh sách các collections
collections_response = client.get_collections()

# Hiển thị danh sách collections
print("Danh sách collections:")
for collection in collections_response.collections:  # Truy cập thuộc tính .collections
    print(f"- {collection.name}")  # Truy cập thuộc tính .name của mỗi collection

In [ ]:
# Lấy danh sách keys từ collection Metachunk_hotpot_Qwen_1.5B_raw
collection_name = "Metachunk_hotpot_Qwen_1.5B_raw"

# Truy vấn tất cả các vector trong collection và lấy danh sách keys
keys = client.scroll(
    collection_name=collection_name,
    scroll_filter=None,  # Không áp dụng filter, lấy toàn bộ
    limit=100  # Số lượng kết quả tối đa trong mỗi lần scroll
)

# In danh sách keys

# print(keys)

Output trong đoạn code này là kết quả từ việc truy vấn một **collection** (bộ sưu tập dữ liệu) có tên `Metachunk_hotpot_Qwen_1.5B_raw`. Định dạng output là một danh sách các đối tượng (objects) kiểu `Record`. Dưới đây là giải thích từng phần chính của cấu trúc output:

---

### 1. **Cấu trúc chính của một `Record`**:
Một `Record` là một đối tượng chứa các thông tin liên quan đến một phần tử dữ liệu trong collection. Các thành phần quan trọng bao gồm:

#### - `id`:
Đây là ID duy nhất để nhận diện phần tử trong collection. Ví dụ:
```plaintext
'id': '000158cf-3459-4587-8e7d-f5f3563aa88a'
```

#### - `payload`:
Đây là thông tin chi tiết của phần tử, bao gồm:
- **`doc`**: ID của tài liệu liên quan đến phần tử.
- **`number_chunk`**: Số thứ tự của chunk (đoạn) dữ liệu trong tài liệu gốc. 
- **`_node_content`**: Đây là nội dung dữ liệu được lưu trữ, thường ở định dạng JSON. Ví dụ:
  ```json
  {
    "id_": "000158cf-3459-4587-8e7d-f5f3563aa88a",
    "metadata": {"doc": 1622, "number_chunk": 18},
    "text": "Dame Margaret Hodge conducted an inquiry into the planned structure..."
  }
  ```
  - `text`: Nội dung chính của chunk (một đoạn văn bản).
  - `metadata`: Metadata đi kèm, ví dụ `doc` (tài liệu) và `number_chunk` (chunk số mấy).
  - `relationships`: Các liên kết (nếu có) với các node khác.

#### - `vector`:
Thông thường đây là giá trị vector hóa của dữ liệu, nhưng trong ví dụ này nó là `null` (không có vector được trả về).

---

### 2. **Danh sách các `Record`**:
Kết quả trả về là danh sách các `Record`, mỗi `Record` đại diện cho một đoạn dữ liệu trong collection. Ví dụ, một đoạn từ kết quả:

```json
{
  "id": "000158cf-3459-4587-8e7d-f5f3563aa88a",
  "payload": {
    "doc": 1622,
    "number_chunk": 18,
    "_node_content": {
      "id_": "000158cf-3459-4587-8e7d-f5f3563aa88a",
      "metadata": {"doc": 1622, "number_chunk": 18},
      "text": "Dame Margaret Hodge conducted an inquiry into the planned structure..."
    }
  }
}
```

Mỗi phần tử có một nội dung văn bản (`text`) kèm theo metadata (ví dụ: `doc`, `number_chunk`) để dễ dàng quản lý và truy xuất dữ liệu.

---

### 3. **Ý nghĩa của cấu trúc này**:
Cấu trúc này được sử dụng để lưu trữ và truy xuất dữ liệu từ các hệ thống quản lý thông tin dạng vector, thường là cho các bài toán AI như tìm kiếm thông minh, hệ thống gợi ý, hoặc xử lý ngôn ngữ tự nhiên.

---

Nếu bạn cần thêm chi tiết hoặc giải thích phần cụ thể nào khác, cứ hỏi nhé! 😊

In [ ]:
# Lấy danh sách keys từ collection
collection_name = "Metachunk_hotpot_Qwen_1.5B_raw"
response = client.scroll(
    collection_name=collection_name,
    scroll_filter=None,  # Không áp dụng filter, lấy toàn bộ
    limit=1000  # Số lượng lớn để đảm bảo không bị thiếu
)

# In kiểu dữ liệu và nội dung
print(type(response))  # tuple 
# Kiểm tra nội dung của tuple
print(response[0])  # Xem phần tử đầu tiên
# print(response[1])  # Xem phần tử thứ hai (nếu có)

In [ ]:
# Lấy tất cả data
collection_name = "Metachunk_hotpot_Qwen_1.5B_raw"
all_data = []  # Lưu tất cả các bản ghi
next_offset = None  # Offset ban đầu là None

while True:
    # Cuộn qua collection
    response = client.scroll(
        collection_name=collection_name,
        scroll_filter=None,  # Không áp dụng filter
        limit=1000,          # Số lượng bản ghi tối đa trong mỗi lần cuộn
        offset=next_offset   # Offset từ lần cuộn trước
    )
    
    # Lấy danh sách bản ghi và offset tiếp theo
    records, next_offset = response
    
    # Thêm bản ghi vào danh sách tổng
    all_data.extend(records)
    
    # Dừng nếu không còn offset tiếp theo (hết dữ liệu)
    if next_offset is None:
        break

# Sau khi hoàn tất, `all_data` chứa toàn bộ dữ liệu từ collection
print(len(all_data))

In [ ]:
print(len(all_data))# sort theo doc (từ 0)
# sort theo number_chunk (từ 0)

# Sắp xếp dữ liệu theo `doc` và `number_chunk`
sorted_data = sorted(all_data, key=lambda x: (x.payload["doc"], x.payload["number_chunk"]))

# Lấy `_node_content` từ dữ liệu đã sắp xếp
detail_result = [json.loads(item.payload["_node_content"]) for item in sorted_data[:5]]
result = [json.loads(item.payload["_node_content"])['text'] for item in sorted_data[:5]]

# In kết quả đã sắp xếp
print(detail_result[0])
print("-----------")
# print(result)

In [ ]:
# Lấy các trường `doc`, `number_chunk`, và `text` từ dữ liệu đã sắp xếp
result = [
    {
        "doc": json.loads(item.payload["_node_content"])["metadata"]["doc"],
        "number_chunk": json.loads(item.payload["_node_content"])["metadata"]["number_chunk"],
        "text": json.loads(item.payload["_node_content"])["text"]
    }
    # for item in sorted_data[:5]  # Giới hạn lấy 5 phần tử đầu
    for item in sorted_data  
]

# In kết quả
print(result)
for item in result:
    print(item)
    # print(f"Doc: {item['doc']}, Number_Chunk: {item['number_chunk']}, Text: {item['text'][:100]}...")  # Text chỉ in 100 ký tự đầu

In [ ]:
# file_path = 'chunk_raptor_part_1.txt'

file_path = 'chunks_data_meta_200token.json'

# Mở file và ghi dữ liệu vào dưới dạng JSON
with open(file_path, 'w') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

print(f"Đã xuất dữ liệu vào file {file_path}")